In [2]:
import pandas as pd
import numpy as np
from  keras.models import Sequential
from keras.layers import Dense, Activation
import keras
from keras.models import load_model
import pickle
import os
import time
from sklearn.utils import shuffle

In [3]:
dirs = os.listdir('./aclImdb/train')

In [4]:
text = []
labels = []
for i in dirs:
    if i == 'neg':
        for j in os.listdir('./aclImdb/train/neg'):
            with open('./aclImdb/train/neg/'+j, 'rb') as f:
                text.append(f.read())
                labels.append([1,0])
    if i == 'pos':
        for j in os.listdir('./aclImdb/train/pos'):
            with open('./aclImdb/train/pos/'+j, 'rb') as f:
                text.append(f.read())
                labels.append([0,1])

In [7]:
my_set = set()
for x in text:
    for i in str(x).split(' '):
        my_set.add(i)

In [8]:
len(my_set)


295309

In [9]:
#key-value pairs for dictionary
word_dict = {}
for index, word in enumerate(my_set):
    word_dict[word] = index

In [10]:
with open('word_dict.pkl', 'wb') as f:
    pickle.dump(word_dict, f)

In [13]:
train_vec = []
for sentence in text[:1000]:
    word_vec = np.zeros(len(my_set))
    for word in str(sentence).split(' '):
        if not word == '':
            word_vec[word_dict[word]] = 1
    train_vec.append(word_vec)
train_vec = np.array(train_vec)

In [14]:
def encode_percep(text, word_dict):
    for sentence in text:
        word_vec = np.zeros(len(my_set))
        for word in str(sentence).split(' '):
            if not word == '':
                word_vec[word_dict[word]] = 1
        train_vec.append(word_vec)
        return np.array(train_vec)

In [23]:
reviews, labels = shuffle(train_vec, labels[:1000])

In [17]:
model = Sequential([
    Dense(120, input_shape=(len(my_set),)),
    Activation('relu'),
    Dense(240),
    Activation('relu'),
    Dense(2),
    Activation('softmax')
])

In [18]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [25]:
model.fit(reviews, np.array(labels), epochs=10, batch_size=32)

Epoch 1/10
1000/1000 [==============================] - 28s 28ms/step - loss: 1.2034e-07 - acc: 1.0000
Epoch 2/10
1000/1000 [==============================] - 27s 27ms/step - loss: 1.2010e-07 - acc: 1.0000
Epoch 3/10
1000/1000 [==============================] - 26s 26ms/step - loss: 1.1986e-07 - acc: 1.0000
Epoch 4/10
1000/1000 [==============================] - 27s 27ms/step - loss: 1.1975e-07 - acc: 1.0000
Epoch 5/10
1000/1000 [==============================] - 27s 27ms/step - loss: 1.1969e-07 - acc: 1.0000
Epoch 6/10
1000/1000 [==============================] - 27s 27ms/step - loss: 1.1957e-07 - acc: 1.0000
Epoch 7/10
1000/1000 [==============================] - 27s 27ms/step - loss: 1.1957e-07 - acc: 1.0000
Epoch 8/10
1000/1000 [==============================] - 27s 27ms/step - loss: 1.1951e-07 - acc: 1.0000
Epoch 9/10
1000/1000 [==============================] - 27s 27ms/step - loss: 1.1939e-07 - acc: 1.0000
Epoch 10/10
1000/1000 [==============================] - 27s 27ms/step - 

In [19]:
score = model.evaluate(train_vec, one_hot_labels[:1000], batch_size=32)

1000/1000 [==============================] - 0s 487us/step


In [20]:
score

[0.3323592805862427, 0.864]

In [26]:
model.save('percept_1.h5')

In [21]:
model.predict(train_vec)

array([[1.0000000e+00, 3.2202116e-11],
       [1.0000000e+00, 2.3910087e-38],
       [1.0000000e+00, 2.7037752e-13],
       ...,
       [1.0000000e+00, 7.0378564e-11],
       [1.0000000e+00, 2.6962412e-09],
       [1.0000000e+00, 1.8600043e-15]], dtype=float32)